In [1]:
from moviepy.editor import AudioFileClip, VideoClip, ImageClip
from sqlalchemy import create_engine
from datetime import datetime
import pandas as pd
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

# Your database connection
engine = create_engine('mysql+pymysql://root@localhost:3306/music_development')
data_path = '../data/'

# Get song data
sql = '''
SELECT S.id, rank, name, artist_id, A.first_name, A.last_name , youtube_code 
FROM songs S
JOIN artists A
ON S.artist_id = A.id
WHERE LOWER(TRIM(name)) LIKE LOWER(TRIM('Million Reasons'))
'''
songs = pd.read_sql(sql, engine)
songs

,id,rank,name,artist_id,first_name,last_name,youtube_code
0,630,1138,Million Reasons,262,Lady,Gaga,en2D_5TzXCA


In [2]:
songId = songs['id'].iloc[0]
songId

630

In [3]:
sql = f'''
SELECT S.id, rank, name, artist_id, A.first_name, A.last_name , youtube_code 
FROM songs S
JOIN artists A
ON S.artist_id = A.id
WHERE S.id = {songId}
'''
song = pd.read_sql(sql, engine)
song

,id,rank,name,artist_id,first_name,last_name,youtube_code
0,630,1138,Million Reasons,262,Lady,Gaga,en2D_5TzXCA


In [4]:
# Get artist name as a string (not pandas Series)
artist_name = f"{song['first_name'].iloc[0]} {song['last_name'].iloc[0]}"
print(f"🎤 Artist: {artist_name}")

🎤 Artist: Lady Gaga


In [5]:
# Get lyrics
sql = f"""
SELECT id, song_id, content 
FROM lyrics 
WHERE song_id = {songId}
"""
lyrics = pd.read_sql(sql, engine)
lyrics

,id,song_id,content
0,556,630,5|(ambient music)\r\n23|(ambient music)\r\n50|...


In [6]:
# Get the lyrics content
lyrics_content = lyrics.content.iloc[0]

# Print with proper line breaks (replace \r\n with actual newlines)
print("Full lyrics content:")
print("-" * 50)
print(lyrics_content.replace('\r\n', '\n'))
print("=" * 50)

Full lyrics content:
--------------------------------------------------
5|(ambient music)
23|(ambient music)
50|("million reasons" piano intro)
54|
64|You're giving me a million 
66|reasons to let you go
68|You're giving me a million 
70|reasons to quit the show
72|You're givin' me a million reasons
74|Give me a million reasons 
76|givin' me a million reasons
78|About a million reasons
79|If I had a highway, I would run for the hills
83|If you could find a dry way, I'd forever be still
86|But you're giving me a million reasons
89|Give me a million reasons 
91|givin' me a million reasons
94|I bow down to pray
99|I try to make the worst seem better
101|Lord, show me the way
106|To cut through all this worn out leather
109|I've got a hundred million 
112|reasons to walk away
116|But baby, I just need one good one to stay

123|Head stuck in a cycle, I look off and I stare
127|It's like that I've stopped breathing, 
129|but completely aware
130|'cause you're giving me a million reasons
133|

In [7]:
# begin of modified function
def parse_timestamp_lyrics(lyrics_text):
    """Parse lyrics with timestamp format from YouTube transcripts using pipe separator"""
    lines = [line.strip() for line in lyrics_text.split('\n') if line.strip()]
    
    lyrics_with_timing = []
    
    for line in lines:
        # Split by the first pipe to separate timestamp from text
        if '|' in line:
            timestamp_str, text = line.split('|', 1)
            try:
                # Convert timestamp to float (seconds)
                timestamp = float(timestamp_str.strip())
                lyrics_with_timing.append({
                    'timestamp': timestamp,
                    'text': text.strip(),  # MODIFIED: Convert to uppercase
                    'start_time': timestamp,
                    'end_time': None,  # Will be calculated later
                    'duration_seconds': None,  # NEW: Will be calculated later
                    'word_count': None,  # NEW: Will be calculated
                    'spaces_count': None,  # NEW: Will be calculated
                    'char_count': None  # NEW: Will be calculated
                })
            except ValueError:
                # Skip lines that don't have valid timestamps
                print(f"⚠️ Skipping line with invalid timestamp: {line}")
                continue
        else:
            print(f"⚠️ Skipping line without Pipe separator: {line}")
    
    # Sort by timestamp
    lyrics_with_timing.sort(key=lambda x: x['timestamp'])
    
    # Calculate end times and additional metrics
    for i in range(len(lyrics_with_timing)):
        if i < len(lyrics_with_timing) - 1:
            lyrics_with_timing[i]['end_time'] = lyrics_with_timing[i + 1]['timestamp']
        else:
            # For the last line, we'll set a default duration
            lyrics_with_timing[i]['end_time'] = lyrics_with_timing[i]['timestamp'] + 5  # 5 seconds default
        
        # NEW: Calculate duration in seconds
        lyrics_with_timing[i]['duration_seconds'] = lyrics_with_timing[i]['end_time'] - lyrics_with_timing[i]['start_time']
        
        # NEW: Calculate word count (split by spaces and filter out empty strings)
        words = [word for word in lyrics_with_timing[i]['text'].split() if word]
        lyrics_with_timing[i]['word_count'] = len(words)
        
        # NEW: Calculate spaces between words (word_count - 1, minimum 0)
        lyrics_with_timing[i]['spaces_count'] = max(0, lyrics_with_timing[i]['word_count'] - 1)
        
        # NEW: Calculate total characters including spaces
        lyrics_with_timing[i]['char_count'] = len(lyrics_with_timing[i]['text'])
    
    return lyrics_with_timing
# end of modified function
    
def get_current_lyric(current_time, lyrics_with_timing):
    """Find which lyric should be displayed at current time"""
    for lyric in lyrics_with_timing:
        if lyric['start_time'] <= current_time < lyric['end_time']:
            return lyric['text']
    return None

def create_timestamp_synced_video(song_id=songId, max_duration=None):
    """Create video using timestamp-based lyrics from YouTube transcripts"""
    
    try:
        # Get song data - use %s parameter syntax for PyMySQL
        query = """
        SELECT s.name as song_name, s.location as audio_file,
               l.content as lyrics, a.first_name, a.last_name
        FROM songs s 
        JOIN lyrics l ON s.id = l.song_id 
        JOIN artists a ON s.artist_id = a.id 
        WHERE s.id = %s
        """
        
        df = pd.read_sql(query, engine, params=(song_id,))
        
        if df.empty:
            print(f"❌ No song found with ID: {song_id}")
            return None
            
        song_data = df.iloc[0]
        
        print(f"🎵 Creating TIMESTAMP-SYNCED video for: {song_data['song_name']}")
        
        # Construct file paths
        audio_dir = os.path.join(r"C:\ruby\music\public\uploads\song\location", str(song_id))
        
        # Debug: Check what's in the directory
        if os.path.exists(audio_dir):
            print(f"📁 Directory contents: {os.listdir(audio_dir)}")
        else:
            print(f"❌ Directory doesn't exist: {audio_dir}")
            return None
            
        audio_path = os.path.join(audio_dir, song_data['audio_file'])
        
        print(f"🔊 Audio path: {audio_path}")
        print(f"🔊 Audio file from DB: {song_data['audio_file']}")

        # Add this line after constructing audio_path
        background_image_path = os.path.join(audio_dir, "Folder.jpg")
        print(f"🖼️ Background image: {background_image_path}")        
        
        # Check if file exists
        if not os.path.exists(audio_path):
            print(f"❌ Audio file not found at: {audio_path}")
            # Try to find the file with different name variations
            all_files = os.listdir(audio_dir)
            print(f"📁 Available files: {all_files}")
            # Look for files containing parts of the name
            for file in all_files:
                if "I_Don" in file and "Want_to_Talk" in file:
                    print(f"🔍 Found alternative file: {file}")
                    audio_path = os.path.join(audio_dir, file)
                    break
            else:
                return None
        
        # Load audio clip to get duration
        audio_clip = AudioFileClip(audio_path)
        full_duration = audio_clip.duration
        
        # Apply duration limit
        if max_duration:
            duration = min(max_duration, full_duration)
            print(f"⏱️ Using LIMITED duration: {duration:.1f}s (max_duration={max_duration}s)")
        else:
            duration = full_duration
            print(f"⏱️ Using FULL duration: {duration:.1f}s")
        
        # Trim audio if needed
        if max_duration and full_duration > max_duration:
            audio_clip = audio_clip.subclip(0, duration)
        
        # Parse timestamp-based lyrics
        lyrics_with_timing = parse_timestamp_lyrics(song_data['lyrics'])
        
        if not lyrics_with_timing:
            print("❌ Could not parse timestamp lyrics")
            return None
        
        print(f"📝 Parsed {len(lyrics_with_timing)} timestamped lyrics lines")
        
        # Filter lyrics to only include those within the limited duration
        if max_duration:
            lyrics_with_timing = [lyric for lyric in lyrics_with_timing if lyric['start_time'] < duration]
            # Adjust the end time of the last lyric to match the limited duration
            if lyrics_with_timing and lyrics_with_timing[-1]['end_time'] > duration:
                lyrics_with_timing[-1]['end_time'] = duration
        
        print(f"📝 Final timing: {len(lyrics_with_timing)} lyrics lines")
        print(f"⏱️ Video duration: {duration:.1f}s ({duration/60:.1f} minutes)")


        # Display parsed timing with new columns
        print("\n📋 Parsed Lyrics Timing:")
        print("-" * 80)
        for i, lyric in enumerate(lyrics_with_timing):
            display_text = lyric['text'][:40] + ('...' if len(lyric['text']) > 40 else '') #no. of chars per line
    
            print(f"{i+1:2d}. {lyric['start_time']:5.1f}s - {lyric['end_time']:5.1f}s: "
                  f"{display_text:<43} | "  # <-- This ensures fixed width of 43 characters
                  f"{lyric['duration_seconds']:4.1f}s | "
                  f"{lyric['word_count']:2d} | "
                  f"{lyric['spaces_count']:2d} | "
                  f"{lyric['char_count']:3d}")
        # end of Display parsed timing with new columns

        # Video settings
        fps = 24
        width, height = 640, 480
        
        def make_synced_frame(t):
            try:
                # Load background
                if os.path.exists(background_image_path):
                    bg_image = Image.open(background_image_path)
                    bg_image = bg_image.resize((width, height), Image.Resampling.LANCZOS)
                    frame = np.array(bg_image)
                else:
                    frame = np.full((height, width, 3), [40, 40, 80], dtype=np.uint8)
                
                # Convert to PIL for text drawing
                pil_img = Image.fromarray(frame)
                draw = ImageDraw.Draw(pil_img)
                
                # Load font
                try:
                    font = ImageFont.truetype("arial.ttf", 32)
                except:
                    try:
                        font = ImageFont.truetype("C:/Windows/Fonts/arial.ttf", 32)
                    except:
                        font = ImageFont.load_default()
                
                # Get current lyric based on timestamp timing
                current_line = get_current_lyric(t, lyrics_with_timing)
                
                if current_line:
                    # Calculate text position
                    try:
                        bbox = draw.textbbox((0, 0), current_line, font=font)
                    except AttributeError:
                        bbox = draw.textsize(current_line, font=font)
                        bbox = (0, 0, bbox[0], bbox[1])
                    
                    text_width = bbox[2] - bbox[0]
                    text_height = bbox[3] - bbox[1]
                    x = (width - text_width) // 2
                    # y = (height - text_height) // 2
                    y = height - 50  # Position near bottom - the lesser the lower
                    
                    text_color = (255, 255, 255) # White text (txtclr)
                    # text_color = (0, 0, 0)         # Black text
                    # Shadow
                    # draw.text((x+2, y+2), current_line, font=font, fill=shadow_color)
                    # Main text
                    # draw.text((x, y), current_line, font=font, fill=text_color)
                    # Main text only (no shadow, no background rectangle)
                    draw.text((x, y), current_line, font=font, fill=text_color)
                return np.array(pil_img)
                
            except Exception as e:
                print(f"❌ Frame error at {t:.1f}s: {e}")
                return np.zeros((height, width, 3), dtype=np.uint8)
        
        # Create video
        print("🎬 Creating timestamp-synced video frames...")
        video = VideoClip(make_synced_frame, duration=duration)
        video = video.set_audio(audio_clip)
        
        # Export
        output_dir = '../data/videos'
        os.makedirs(output_dir, exist_ok=True)
        
        # Include duration in filename
        duration_suffix = f"_{max_duration}s" if max_duration else "_full"
        
        # Use the artist name from song_data to ensure it's a string
        clean_artist_name = f"{song_data['first_name']} {song_data['last_name']}"
        output_file = os.path.join(output_dir, f"{song_data['song_name']}_{clean_artist_name}.mp4")
        
        print("📹 Exporting video...")
        video.write_videofile(
            output_file, 
            fps=fps, 
            codec='libx264',
            audio_codec='aac',
            verbose=False,
            logger=None
        )
        
        print(f"✅ TIMESTAMP-SYNCED VIDEO CREATED: {output_file}")
        print(f"📊 File size: {os.path.getsize(output_file) / (1024*1024):.1f} MB")
        
        # Clean up
        video.close()
        audio_clip.close()
        
        return output_file
        
    except Exception as e:
        print(f"❌ Video creation error: {e}")
        import traceback
        traceback.print_exc()
        return None

# Run the timestamp-synced version
if __name__ == "__main__":
    print("=" * 70)
    print("🎬 CREATING TIMESTAMP-SYNCED VIDEO")
    print("=" * 70)
    
    result = create_timestamp_synced_video(song_id=songId, max_duration=None)
    # print(output_file)
    if result:
        # print(f"\n🎉 SUCCESS! video created: {output_file}")       
        print(f"\n🎉 SUCCESS! Timestamp-synced video created: {result}")
        print("\n✨ Features:")
        print("   ✅ YouTube transcript timestamp parsing")
        print("   ✅ Pipe separator for timestamps")
        print("   ✅ No complex audio analysis needed")
        print("   ✅ Perfect synchronization with original timestamps")
        
    else:
        print("\n❌ Timestamp-synced video creation failed")

🎬 CREATING TIMESTAMP-SYNCED VIDEO
🎵 Creating TIMESTAMP-SYNCED video for: Million Reasons
📁 Directory contents: ['Folder.jpg', 'Lady_Gaga_-_Million_Reasons.mp3']
🔊 Audio path: C:\ruby\music\public\uploads\song\location\630\Lady_Gaga_-_Million_Reasons.mp3
🔊 Audio file from DB: Lady_Gaga_-_Million_Reasons.mp3
🖼️ Background image: C:\ruby\music\public\uploads\song\location\630\Folder.jpg
⏱️ Using FULL duration: 252.2s
📝 Parsed 65 timestamped lyrics lines
📝 Final timing: 65 lyrics lines
⏱️ Video duration: 252.2s (4.2 minutes)

📋 Parsed Lyrics Timing:
--------------------------------------------------------------------------------
 1.   5.0s -  23.0s: (ambient music)                             | 18.0s |  2 |  1 |  15
 2.  23.0s -  50.0s: (ambient music)                             | 27.0s |  2 |  1 |  15
 3.  50.0s -  54.0s: ("million reasons" piano intro)             |  4.0s |  4 |  3 |  31
 4.  54.0s -  64.0s:                                             | 10.0s |  0 |  0 |   0
 5.  64.0s 